In [1]:
import numpy as np
import csv
import pandas as pd
import random


### Необходимые наборы данных

In [2]:
# среднее количество пациентов, поступающих в час
avg_patients = {
    0: ["00:00", 3.690616],
    1: ["01:00", 2.911858],
    2: ["02:00", 2.293054],
    3: ["03:00", 2.017725],
    4: ["04:00", 1.831175],
    5: ["05:00", 1.856022],
    6: ["06:00", 2.251625],
    7: ["07:00", 3.803911],
    8: ["08:00", 5.446445],
    9: ["09:00", 7.066014],
    10: ["10:00", 7.939452],
    11: ["11:00", 8.49382],
    12: ["12:00", 8.178273],
    13: ["13:00", 7.79489],
    14: ["14:00", 7.792522],
    15: ["15:00", 8.053659],
    16: ["16:00", 7.983501],
    17: ["17:00", 7.969416],
    18: ["18:00", 8.282366],
    19: ["19:00", 7.664413],
    20: ["20:00", 7.238266],
    21: ["21:00", 6.578026],
    22: ["22:00", 5.526836],
    23: ["23:00", 4.336112], 
}

# набор данных с предпочтением бригад скорой помощи на каждую смену 
shifts_preference = {
    0: ["7:00", 8, 6],
    1: ["7:00", 10, 5],
    2: ["7:00", 12, 3],
    3: ["11:00", 8, 6],
    4: ["11:00", 10, 3],
    5: ["11:00", 12, 4],
    6: ["15:00", 8, 6],
    7: ["15:00", 10, 3],
    8: ["15:00", 12, 1],
    9: ["19:00", 8, 4],
    10: ["19:00", 10, 4],
    11: ["19:00", 12, 2],
    12: ["23:00", 8, 2],
    13: ["23:00", 10, 3],
    14: ["23:00", 12, 3],
    15: ["no schedule", 0, 0]
}

# набор данных с временем начала смены и продолжительностью
shifts = {
    0: ["7:00", 8],
    1: ["7:00", 10],
    2: ["7:00", 12],
    3: ["11:00", 8],
    4: ["11:00", 10],
    5: ["11:00", 12],
    6: ["15:00", 8],
    7: ["15:00", 10],
    8: ["15:00", 12],
    9: ["19:00", 8],
    10: ["19:00", 10],
    11: ["19:00", 12],
    12: ["23:00", 8],
    13: ["23:00", 10],
    14: ["23:00", 12],
    15: ["no schedule", 0]
}

### Инициализация популяции

<img src="Image\ch.png" width="500" height="250">

In [3]:
def generate_chromosome(population_size, shifts):

    # Максимальное количество часов должны отработать бригады в день
    max_hours = 48 
    
    # считываются индексы (ключи) из словаря shifts
    available_shifts = list(shifts.keys())
    
    # Создаем пустой список для хранения хромосом
    population = []
    hour_counts = []

    # Генерируем популяцию
    while len(population) < population_size:
        chromosome = []
    
        # Генерируем каждый ген в хромосоме
        for _ in range(6):
            gene_value = random.choice(available_shifts)
            
            # Добавляем ген в хромосому
            chromosome.append(gene_value)
        chromosome.append(0)

        dur = []

        # Подсчитываем общее количество часов для хромосомы
        chromosome_ = chromosome[:-1]
        for gene in chromosome_:  
            start_time, duration = shifts[gene]         
            dur.append(duration)
    
        total_hours_dur = sum(dur)

        # Проверяем превышение максимального количества часов
        if total_hours_dur <= max_hours:

        # Создаем список всех часов в сутках
            all_hours = list(range(24))

            # Создаем временный словарь для подсчета врачей в текущей хромосоме
            temp_hour_count = {hour: 0 for hour in all_hours}

            # Проходим по каждому гену в текущей хромосоме
            for gene_value in chromosome_:
                # Проверяем, если смена с данным номером существует в словаре shifts
                if gene_value in shifts:
                    start_time, duration = shifts[gene_value]  # Получаем время начала смены и ее длительность
                    
                    # Если смена имеет расписание, обновляем подсчет врачей
                    if start_time != "no schedule":
                        start_hour = int(start_time.split(":")[0])  # Получаем час начала смены
                        # Подсчитываем количество врачей в каждом часу смены
                        for hour in range(start_hour, start_hour + duration + 1):
                            temp_hour_count[hour % 24] += 1  # Обработка цикличности часов

            # Создаем пустой список для хранения количества врачей в каждом часу
            hour_count = {hour: 0 for hour in all_hours}

            # Обновляем общий подсчет врачей на основе текущей хромосомы
            for hour in all_hours:
                hour_count[hour] += temp_hour_count[hour]

            # Проверяем, если хотя бы один час имеет 0 свободных врачей, устанавливаем флаг в False
            if 0 in hour_count.values():
                chromosome_valid = False
            else:
                chromosome_valid = True
                
            if chromosome_valid == True:
                population.append(chromosome)
                hour_counts.append(list(hour_count.values()))
    
    return population, hour_counts


In [4]:
def check_population(population, shifts):

    checked_population = []

        # Подсчитываем общее количество часов для хромосомы
    for chromosome in population:
        dur = []
        max_hours = 48 
        # available_shifts = list(shifts.keys())
        population = []
        hour_counts = []

        for gene in chromosome[:-1]:  
            start_time, duration = shifts[gene]         
            dur.append(duration)
    
        total_hours_dur = sum(dur)

        # Проверяем превышение максимального количества часов
        if total_hours_dur <= max_hours:

        # Создаем список всех часов в сутках
            all_hours = list(range(24))

            # Создаем временный словарь для подсчета врачей в текущей хромосоме
            temp_hour_count = {hour: 0 for hour in all_hours}

            # Проходим по каждому гену в текущей хромосоме
            for gene_value in chromosome[:-1]:
                # Проверяем, если смена с данным номером существует в словаре shifts
                if gene_value in shifts:
                    start_time, duration = shifts[gene_value]  # Получаем время начала смены и ее длительность
                    
                    # Если смена имеет расписание, обновляем подсчет врачей
                    if start_time != "no schedule":
                        start_hour = int(start_time.split(":")[0])  # Получаем час начала смены
                        # Подсчитываем количество врачей в каждом часу смены
                        for hour in range(start_hour, start_hour + duration + 1):
                            temp_hour_count[hour % 24] += 1  # Обработка цикличности часов

            # Создаем пустой список для хранения количества врачей в каждом часу
            hour_count = {hour: 0 for hour in all_hours}

            # Обновляем общий подсчет врачей на основе текущей хромосомы
            for hour in all_hours:
                hour_count[hour] += temp_hour_count[hour]

            # Проверяем, если хотя бы один час имеет 0 свободных врачей, устанавливаем флаг в False
            if 0 in hour_count.values():
                chromosome_valid = False
            else:
                chromosome_valid = True
                
            if chromosome_valid == True:
                checked_population.append(chromosome)
                hour_counts.append(list(hour_count.values()))
    return checked_population

##### Проверка на работоспособность функции инициализации

In [5]:
# population, hour_counts = generate_chromosome(10, shifts)
# for chromosome in population:
#     print(chromosome)

### Вычисление фитнесс - функции


In [6]:
def fitness(population, hour_counts):
    
    mean_pat = [shifts[1] for shifts in avg_patients.values()] # берем из словаря "avg_patients" среднее количество пациентов в очереди
    pj = [shift[2] for shift in shifts_preference.values()] # берем из словаря "shifts_preference"

    z_fit = np.zeros(len(population)) 
    min_value = np.inf
    best_chromosome_ = None

    for i, chromosome in enumerate(population):
        chromosome_ = chromosome[:-1]

        #1-ое слагаемое в функции 

        w_list = []
        w_penalty = []

        for hours in range(24):

            lamb = np.random.poisson(mean_pat[hours])
            mu = np.clip(np.random.exponential(15, size=1), 0.5, np.inf)
            s = hour_counts[hours]
            w = (np.array(lamb))/(s*np.abs(mu-np.array(lamb)))
            
            w_list.append(w)
            w_penalty.append(np.abs(mu - 15))

        sum_w = np.sum(w_list) 
        sum_omega_1 = np.sum(w_penalty)

        norm_omega_1 = (sum_omega_1 - np.min(w_penalty))/(np.max(w_penalty) - np.min(w_penalty))

        res_1 = norm_omega_1 * sum_w

        # Вычисление 2-ого слагаемого

        omega_2_list = []
        p_list = []

        for gene in chromosome_:

            if gene == 15:
                omega_2 = 0
                omega_2_list.append(omega_2)
            else:
                omega_2 = 6 - pj[gene]
                omega_2_list.append(omega_2)
            
            sum_omega_2 = np.sum(omega_2_list)
            omega_2_norm = (sum_omega_2 - np.min(omega_2_list))/(np.max(omega_2_list) - np.min(omega_2_list))

            p_list.append(pj[gene])

        sum_p = np.sum(p_list)

        res_2 = sum_p * omega_2_norm

        # Вычисление значения z
        z = res_1 + res_2

        chromosome[6] = z

        if z < min_value:
            min_value = z
        
            if chromosome[6] == min_value:
                best_chromosome_ = chromosome_
        
        z_fit[i] = z

    return population, z_fit, min_value, best_chromosome_

##### Проверка работоспособности фитнесс функции

In [7]:
# population, hour_counts = generate_chromosome(population_size= 10, shifts = shifts)
# population1, z_fit, min_value, best_chromosome_ = fitness(population, hour_counts= hour_counts)

# print(population1)
# print(z_fit)

### Селекция

In [8]:
def selection(population):

    sorted_lists = sorted(population, key=lambda x: x[6], reverse = False)

    selected_lists = sorted_lists[0:int(len(sorted_lists) * 0.8)]


    no_parents = int(len(selected_lists) / 2) 
    indx = list(range(len(selected_lists))) 
    random.shuffle(indx)  
    
    group1 = indx[:no_parents]
    group2 = indx[no_parents:]

    selected_pairs = [] 
    for i in range(no_parents):
        parent1 = random.choice(group1)
        parent2 = random.choice(group2)
        selected_pairs.append(selected_lists[parent1])
        selected_pairs.append(selected_lists[parent2])

        group1.remove(parent1)
        group2.remove(parent2)
    

    return selected_pairs

### Скрещивание

<img src="Image\cross.png" width="500" height="200">

In [9]:
def swapgenes(chromosome1, chromosome2, crossover_point,):

    child1 = chromosome1.copy()
    child2 = chromosome2.copy()

    # Обмен значений генов на заданной точке пересечения
    for i in range(crossover_point + 1):
        child1[i] = chromosome2[i]
        child2[i] = chromosome1[i]

    return child1, child2

def crossover(selected, crossover_point):
    children = []

    for i in range(0, len(selected), 2):
        child1, child2 = swapgenes(selected[i], selected[i + 1], crossover_point)  # Распаковка кортежа
        children.append(child1)
        children.append(child2)

    return children

def check(children, selected_pairs, shifts):
    # check_children_ = check_children

    check_child = check_population(children, shifts)
    diff = len(children) - len(check_child)
    plus_child = random.choices(selected_pairs, k = diff)

    # check_children_ = check_children
    check_child.extend(plus_child)


    return check_child



    

In [10]:
population, hour_counts = generate_chromosome(population_size= 20, shifts= shifts)
selected = selection(population)
children = crossover(selected, crossover_point= 2)
print(children)
print(len(children))
chiks = check(children, selected, shifts)


[[15, 1, 13, 11, 0, 15, 0], [6, 7, 12, 15, 14, 6, 0], [7, 12, 0, 15, 9, 13, 0], [1, 9, 6, 1, 15, 15, 0], [12, 3, 12, 1, 6, 4, 0], [12, 15, 6, 0, 15, 7, 0], [10, 3, 2, 14, 15, 1, 0], [15, 5, 2, 12, 3, 15, 0], [11, 15, 13, 6, 15, 2, 0], [13, 15, 14, 8, 15, 1, 0], [6, 2, 3, 9, 12, 3, 0], [0, 15, 13, 4, 13, 15, 0], [1, 6, 12, 15, 2, 10, 0], [10, 12, 6, 15, 6, 15, 0], [10, 12, 4, 13, 0, 12, 0], [12, 5, 15, 9, 2, 15, 0]]
16


In [11]:
population, hour_counts = generate_chromosome(population_size= 50, shifts= shifts)
selected = selection(population)
children = crossover(selected, crossover_point= 2)
checked = check_population(children, shifts = shifts)
print(children)
print(checked)

[[13, 3, 15, 12, 15, 11, 0], [0, 8, 3, 13, 6, 1, 0], [11, 14, 15, 9, 12, 15, 0], [8, 2, 12, 7, 2, 15, 0], [9, 11, 15, 3, 7, 13, 0], [15, 13, 0, 0, 9, 7, 0], [0, 7, 3, 0, 2, 15, 0], [11, 3, 0, 12, 3, 15, 0], [3, 12, 11, 9, 12, 15, 0], [1, 5, 15, 15, 9, 0, 0], [14, 15, 9, 3, 9, 0, 0], [15, 13, 4, 9, 9, 2, 0], [2, 12, 4, 15, 6, 2, 0], [11, 9, 9, 15, 12, 10, 0], [1, 13, 7, 4, 15, 5, 0], [14, 8, 15, 12, 10, 15, 0], [15, 6, 7, 4, 15, 6, 0], [1, 12, 7, 1, 12, 12, 0], [12, 6, 15, 15, 12, 9, 0], [6, 12, 1, 10, 0, 6, 0], [14, 1, 9, 6, 7, 11, 0], [7, 0, 15, 12, 6, 15, 0], [14, 10, 3, 6, 12, 12, 0], [2, 15, 10, 15, 0, 3, 0], [10, 15, 3, 15, 11, 6, 0], [0, 6, 0, 14, 2, 15, 0], [3, 9, 13, 15, 1, 14, 0], [6, 6, 10, 15, 6, 14, 0], [0, 15, 11, 0, 6, 8, 0], [15, 14, 6, 6, 10, 7, 0], [14, 5, 3, 0, 9, 9, 0], [15, 11, 6, 12, 15, 3, 0], [12, 9, 2, 11, 8, 15, 0], [1, 7, 15, 0, 2, 15, 0], [9, 15, 1, 14, 3, 9, 0], [4, 15, 1, 12, 13, 8, 0], [9, 12, 14, 3, 9, 15, 0], [14, 9, 15, 4, 15, 12, 0], [15, 0, 7, 3, 15, 

### Проверка работоспособности функции "crossover"

In [12]:
# pop, hour_counts = generate_chromosome(population_size= 10, shifts= shifts)
# pop1 = fitness(pop)
# selected = selection(pop1)
# children = crossover(selected, crossover_point= 2)

# print(f'Выбранные родители: {selected};  Дети: {children}')


### Реализация генетического алгоритма

In [13]:
def GA_Find_GlobalMinima(population_size, no_generations):

    population, hour_counts = generate_chromosome(population_size, shifts=shifts)
    best_chrom = None
    best_fitness = float('inf')
    
    for generation in range(no_generations):
        populate, z_fit, min_value, best = fitness(population, hour_counts)   
        selected_pairs = selection(populate) 
        newpopulation = crossover(selected_pairs, 2) 
        chiks = check(newpopulation, selected_pairs, shifts)
        population = chiks
        
        # Обновление лучшей хромосомы
        if min_value < best_fitness:
            best_chrom = np.append(best, min_value)
            best_fitness = min_value
             
    return best_chrom, best_fitness


# population, hour_counts = generate_chromosome(population_size= 20, shifts= shifts)
# selected = selection(population)
# children = crossover(selected, crossover_point= 2)
# print(children)
# print(len(children))
# chiks = check(children, selected, shifts)

##### Проверка работоспособности генетического алгоритма

In [14]:
best_chrom1, best_fit = GA_Find_GlobalMinima(population_size= 1000, no_generations= 2000)
print(best_chrom1)
print(best_fit)


C:\Temp\ipykernel_19372\148149161.py:50: RuntimeWarning: invalid value encountered in scalar divide
  omega_2_norm = (sum_omega_2 - np.min(omega_2_list))/(np.max(omega_2_list) - np.min(omega_2_list))
C:\Temp\ipykernel_19372\148149161.py:50: RuntimeWarning: divide by zero encountered in scalar divide
  omega_2_norm = (sum_omega_2 - np.min(omega_2_list))/(np.max(omega_2_list) - np.min(omega_2_list))


[  0.           5.          15.          12.           1.
   4.         842.97177178]
842.9717717830775


### Вывод: в ходе выполнения данной работы, мы получили хромосому (график смены), которая удовлетворяет заданным ограничениям: В сумме продолжительность смен в день не должна превышать 48 часов
### В каждый час должна быть хотя бы 1 свободная бригада. 

{0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 3, 8: 2, 9: 2, 10: 2, 11: 4, 12: 4, 13: 4, 14: 4, 15: 4, 16: 3, 17: 3, 18: 2, 19: 2, 20: 2, 21: 2, 22: 1, 23: 2}


shifts = {
    0: ["7:00", 8],
    1: ["7:00", 10],
    2: ["7:00", 12],
    3: ["11:00", 8],
    4: ["11:00", 10],
    5: ["11:00", 12],
    6: ["15:00", 8],
    7: ["15:00", 10],
    8: ["15:00", 12],
    9: ["19:00", 8],
    10: ["19:00", 10],
    11: ["19:00", 12],
    12: ["23:00", 8],
    13: ["23:00", 10],
    14: ["23:00", 12],
    15: ["no schedule", 0]
}

shifts_preference = {
    0: ["7:00", 8, 6],
    1: ["7:00", 10, 5],
    2: ["7:00", 12, 3],
    3: ["11:00", 8, 6],
    4: ["11:00", 10, 3],
    5: ["11:00", 12, 4],
    6: ["15:00", 8, 6],
    7: ["15:00", 10, 3],
    8: ["15:00", 12, 1],
    9: ["19:00", 8, 4],
    10: ["19:00", 10, 4],
    11: ["19:00", 12, 2],
    12: ["23:00", 8, 2],
    13: ["23:00", 10, 3],
    14: ["23:00", 12, 3],
    15: ["no schedule", 0, 0]
}

[  0.           5.          15.          12.           1.
   4.         842.97177178]

### Список использованных источников:

- https://core.ac.uk/download/pdf/268766133.pdf